Here:
- generate time series (cont OU process?)
- plot it (continious and also binned)
- calculate acf: using ABC tau, using py functions
- plot acf

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns # comment this line if you don't want to use seaborn for plots

import numpy as np
# IP import warnings because of "module 'numpy' has no attribute 'warnings'"
import warnings
np.warnings = warnings
import pandas as pd
import statsmodels.api as sm
from scipy import stats

# add the path to the abcTau package
import sys
#sys.path.append('./abcTau')
sys.path.append('C:\\Users\\ipochino\\.conda\\envs\\isttc\\Lib\\site-packages\\abcTau') # IP: replaced previous line with that; relative path was not working
import abcTau

In [ ]:
def OU_gen(tau, D, deltaT, T, numTrials):
    """Generate an OU process with a single timescale, zero mean and unit variance.

    Parameters
    -----------
    tau : float
        timescale.
    D : float
        diffusion parameter.
    deltaT : float
        temporal resolution for the OU process generation.    
    T : float
        duration of trials.
    numTrials : float
        number of trials.
    
    
    Returns
    -------
    ou : nd array
        array of generated OU process (numTrials * (T/deltaT)).
    """
    
    numBin = int(T/deltaT)
    noise =  np.random.normal(loc=0,scale=1, size=(numTrials,numBin))
    ou = np.zeros((numTrials,numBin))
    ou[:,0] = noise[:,0]
    for iBin in range(1,numBin):
        ou[:,iBin]  = ou[:,iBin-1] - (ou[:,iBin-1]/tau) * deltaT + np.sqrt(2*D*deltaT) * noise[:,iBin-1]
        
    return ou
# IP comments:
# 1. not sure why we multiply by deltaT in diffusion part - the finer the resolution (higher numBin) the less diffusion we have - this sort 
# of make sense

### Generate time series

In [ ]:
tau = np.array([100])
D = 1/tau
deltaT = 1
T = 1000
numTrials = 1

# generate OU
#ou_all = OU_gen(tau, D, deltaT, T, numTrials)

In [ ]:
# generate OU
numBin = int(T/deltaT)
print('numBin:', numBin)

noise =  np.random.normal(loc=0,scale=1, size=(numTrials,numBin))
print('noise.shape: ', noise.shape)

noise_ou = np.sqrt(2*D*deltaT) * noise
print('noise_ou.shape: ', noise_ou.shape)

ou = np.zeros((numTrials,numBin))
ou[:,0] = noise[:,0]
for iBin in range(1,numBin):
    ou[:,iBin]  = ou[:,iBin-1] - (ou[:,iBin-1]/tau) * deltaT + np.sqrt(2*D*deltaT) * noise[:,iBin-1]

print('ou.shape: ', ou.shape)

In [ ]:
# adjust OU process - scale, shift and rectify (to get time-varying rate)

data_mean = 1 # average of firing rate
data_var = 1.5 # variance of firing rate

ou_check = np.max(ou)
print(ou_check)

ou_std = np.sqrt(data_var)
ou_mean = data_mean
ou_all = ou_std * ou + ou_mean # this is scale and shift, where is rectify?

print('ou_all.shape: ', ou_all.shape)

In [ ]:
binSize = 50 # bins size for binning the data for calculating acf
binsData =  np.arange(0, T + binSize, binSize)
numBinData = len(binsData)-1
print(binsData)
print(numBinData)

In [ ]:
new_shape = [numTrials, numBinData]
shape = (new_shape[0], ou_all.shape[0] // new_shape[0],
         new_shape[1], ou_all.shape[1] // new_shape[1])
print(shape)
binned_data = ou_all.reshape(shape).sum(-1).sum(1)
print(binned_data.shape)

In [ ]:
binned_data = ou_all.reshape(shape).sum(-1).sum(1)
binned_data.shape
binned_data[0]

In [ ]:
fig, axes = plt.subplots(5,1, figsize=(10,8))

axes[0].plot(np.linspace(0,numBin-1,numBin), noise[0], lw=0.5)
axes[0].axhline(y=0, color='k', lw=0.5)

axes[1].plot(np.linspace(0,numBin-1,numBin), noise_ou[0], lw=0.5)
axes[1].axhline(y=0, color='k', lw=0.5)

axes[2].plot(np.linspace(0,numBin-1,numBin), ou[0], lw=0.5)
axes[2].axhline(y=0, color='k', lw=0.5)

axes[3].plot(np.linspace(0,numBin-1,numBin), ou_all[0], lw=0.5)
axes[3].axhline(y=0, color='k', lw=0.5)
for i in range(numBinData+1):
    axes[3].axvline(x=50*i, color='k', lw=0.5)

# that is supposed to be binned rate but it goes negative? is it ok?
axes[4].plot(np.linspace(0,numBinData-1,numBinData), binned_data[0], lw=0.5)
axes[4].axhline(y=0, color='k', lw=0.5)
axes[4].set_xticks(np.linspace(0,numBinData-1,numBinData))

sns.despine()

In [ ]:
pd.plotting.autocorrelation_plot(ou_all[0,:])

In [ ]:
#binSize = 50
maxTimeLag = 1000 
summStat_metric = 'comp_cc'
ifNorm = True # if normalize the autocorrelation or PSD
data_sumStat, data_mean, data_var, T, numTrials =  abcTau.preprocessing.extract_stats(ou_all, deltaT, binSize,
                                                                                  summStat_metric, ifNorm, maxTimeLag)
print(data_sumStat)

In [ ]:
t_axis_lags = np.linspace(0, 1000, 20)

fig, axes = plt.subplots(1,1, figsize=(7,5))

axes.plot(t_axis_lags, data_sumStat, c='k', label='abc')
axes.plot(t_axis_lags, acf_, c='orange', label='acf_statstool')
pd.plotting.autocorrelation_plot(ou_all[0,:], ax=axes, c='steelblue', label='autocorr_plot')
#pd.plotting.autocorrelation_plot(ou[0,:], ax=axes, c='darkblue', label='autocorr_plot')
axes.axvline(x=100, lw=0.5, color='k')
axes.axhline(y=1/np.e, lw=0.5, color='k')

sns.despine()

In [ ]:
from statsmodels.tsa.stattools import acf
acf_ = acf(binned_data[0], nlags=20)

In [ ]:
popt, poptcov = abcTau.preprocessing.fit_oneTauExponential(data_sumStat, binSize, maxTimeLag)
tau = popt[1]

print(popt)

In [ ]:
from scipy.optimize import curve_fit
def func_exp(x, a, b, c):
    """
    Exponential function to fit the data.
    :param x: 1d array, independent variable
    :param a: float, parameter to fit
    :param b: float, parameter to fit
    :param c: float, parameter to fit
    :return: callable
    """
    return a * np.exp(-b * x) + c
    
popt, pcov = curve_fit(func_exp, np.linspace(0,20,20), acf_[0:], maxfev=5000)
print(popt)

tau = 1 / popt[1]
tau_ms = tau * binSize
print(tau, tau_ms)

In [ ]:
# select generative model and distance function
generativeModel = 'oneTauOU'
distFunc = 'linear_distance'

# Define a uniform prior distribution over the given range
# for a uniform prior: stats.uniform(loc=x_min,scale=x_max-x_min)
t_min = 0.0 # first timescale
t_max = 100.0
priorDist = [stats.uniform(loc= t_min, scale = t_max - t_min)]

# set fitting params
epsilon_0 = 1  # initial error threshold
min_samples = 100 # min samples from the posterior
steps = 60 # max number of iterations
minAccRate = 0.01 # minimum acceptance rate to stop the iterations
parallel = False # if parallel processing
n_procs = 1 # number of processor for parallel processing (set to 1 if there is no parallel processing)

In [ ]:
# creating model object
class MyModel(abcTau.Model):

    #This method initializes the model object.  
    def __init__(self):
        pass

    # draw samples from the prior. 
    def draw_theta(self):
        theta = []
        for p in self.prior:
            theta.append(p.rvs())
        return theta

    # Choose the generative model (from generative_models)
    # Choose autocorrelation computation method (from basic_functions)
    def generate_data(self, theta):
        # generate synthetic data
        if disp == None:
            syn_data, numBinData =  eval('abcTau.generative_models.' + generativeModel + \
                                         '(theta, deltaT, binSize, T, numTrials, data_mean, data_var)')
        else:
            syn_data, numBinData =  eval('abcTau.generative_models.' + generativeModel + \
                                         '(theta, deltaT, binSize, T, numTrials, data_mean, data_var, disp)')
               
        # compute the summary statistics
        syn_sumStat = abcTau.summary_stats.comp_sumStat(syn_data, summStat_metric, ifNorm, deltaT, binSize, T,\
                                          numBinData, maxTimeLag)   
        return syn_sumStat

    # Computes the summary statistics
    def summary_stats(self, data):
        sum_stat = data
        return sum_stat

    # Choose the method for computing distance (from basic_functions)
    def distance_function(self, data, synth_data):
        if np.nansum(synth_data) <= 0: # in case of all nans return large d to reject the sample
            d = 10**4
        else:
            d = eval('abcTau.distance_functions.' +distFunc + '(data, synth_data)')        
        return d

In [ ]:
# path for loading and saving data
datasave_path = 'Q:\\Personal\\Irina\\projects\\isttc\\results\\synthetic_data\\'
inter_save_direc = datasave_path
inter_filename = 'inter_filename'

filename = 'test_1_trial'
filenameSave = filename

disp = None

In [ ]:
# fit with aABC algorithm for any generative model
abc_results, final_step = abcTau.fit.fit_withABC(MyModel, data_sumStat, priorDist, inter_save_direc, inter_filename,\
                                                 datasave_path,filenameSave, epsilon_0, min_samples, \
                                                 steps, minAccRate, parallel, n_procs, disp)

In [ ]:
# extract estimated parameters
theta_accepted = abc_results[final_step-1]['theta accepted']
tau1 = theta_accepted[0]

In [ ]:
tau1

In [ ]:
plt.hist(tau1, density=True, label = r'Estimated')